In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder
import pickle

In [2]:
df = pd.read_csv('Churn_Modelling.csv')

In [3]:
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
data = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [6]:
data.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [7]:
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])

In [8]:
data.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1


In [9]:
data['Gender'].unique()

array([0, 1])

In [10]:
data['Gender'].value_counts()

1    5457
0    4543
Name: Gender, dtype: int64

In [22]:
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()

In [23]:
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [24]:
type(geo_encoder)

numpy.ndarray

In [25]:
onehot_encoder_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [26]:
geography_data = pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [27]:
geography_data.head(3)

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0


In [28]:
updated_data = pd.concat([data.drop('Geography',axis=1),geography_data],axis=1)

In [29]:
updated_data.head(3)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0


In [30]:
x = updated_data.drop(['Exited'],axis=1)
y = updated_data['Exited']

In [31]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [32]:
scaler = StandardScaler()

In [33]:
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [34]:
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

In [35]:
with open('gender_label_encoder.pkl','wb') as f:
    pickle.dump(label_encoder,f)

In [36]:
with open('geo_onehot_encoder.pkl','wb') as f:
    pickle.dump(onehot_encoder_geo,f)

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [59]:
model = Sequential([
    Dense(64,activation=tf.nn.relu,input_shape=(x_train.shape[1],)),
    Dense(32,activation=tf.nn.relu),
    Dense(1,activation=tf.nn.sigmoid)
])

In [60]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.compile(tf.keras.optimizers.Adam(learning_rate=0.001),loss="binary_crossentropy",
             metrics=['accuracy'])

In [15]:
#set_ups the tensorboard

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_data = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [64]:
early_stoping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [67]:
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),
          callbacks=[tensorboard_data,early_stoping])

Epoch 1/100
250/250 [==============================] - 1s 6ms/step - loss: 0.3063 - accuracy: 0.8715 - val_loss: 0.3411 - val_accuracy: 0.8615
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3058 - accuracy: 0.8712 - val_loss: 0.3443 - val_accuracy: 0.8570
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3039 - accuracy: 0.8715 - val_loss: 0.3464 - val_accuracy: 0.8595
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3024 - accuracy: 0.8734 - val_loss: 0.3420 - val_accuracy: 0.8650
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3014 - accuracy: 0.8724 - val_loss: 0.3443 - val_accuracy: 0.8615
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3001 - accuracy: 0.8754 - val_loss: 0.3432 - val_accuracy: 0.8575


In [68]:
model.save('Ann_model.h5')

In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 36764), started 0:18:03 ago. (Use '!kill 36764' to kill it.)

In [4]:
%tensorboard --logdir {logs}  --host localhost

Reusing TensorBoard on port 6006 (pid 100500), started 0:06:19 ago. (Use '!kill 100500' to kill it.)

In [ ]:
20.198.110.60

In [ ]:
20.204..155

In [ ]:
9031